# Solving Relativistic Fluid 2D

### Introduction
The goal of this notebook is to study turbulence from a perfect relativistic fluid. We consider the stress energy tensor for a perfect fluid in Minkowski space,

\begin{equation}
T_{ab} = p\eta_{ab} + (e+p)u_a u_b
\end{equation}

The equations of motion for such fluid are,

\begin{equation}
\label{EOM}
\partial_a T^{ab} = 0. \quad (1)
\end{equation}

Here we focus on a conformal fluid, that is the stress energy tensor is traceless,

$$T_a^a = 0 \to e = -2p.$$

To solve eq.1 numerically, we rewrite the EOM in conservative variables $U=(D,S^i)$ where,

$$\frac{D}{2} = T^{00} = \frac{e}{2}\left(-1+\frac{3}{W^2}\right)$$

$$\frac{S^i}{2} = T^{0i} = \frac{3e}{2}\frac{v^i}{W^2}$$

Where we have the Lorentz factor $W = \sqrt{1-v^iv_i}$. Now the EOM in conservative form,

$$\partial_t D + \partial_i S^i = 0$$
$$\partial_t S^i + \partial_j(e \delta^{ij} + S^i v^j) = 0$$

In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
import math

## Parameters

In [2]:
# Number of grid points
Nx = 401
Ny = 401
Nt = 300

# Discretisation parameters
cfl = 0.9
dx  = 10./(Nx)
dy  = 10./(Ny)
dt  = cfl*min(dx,dy)

# Spacial linspace
x = np.linspace(0,dx*Nx,Nx)
y = np.linspace(0,dy*Ny,Ny)

# physical parameters
gamma = 1.5

# Numerical parameters (floor)
epsilon = 10e-5

In [3]:
# Variables to store solutions
v_x_old = np.zeros((Nx,Ny))
v_x_new = np.zeros((Nx,Ny))

v_y_old = np.zeros((Nx,Ny))
v_y_new = np.zeros((Nx,Ny))

rho_old = np.ones((Nx,Ny))
rho_new = np.ones((Nx,Ny))

D_old = np.zeros((Nx,Ny))
D_new = np.zeros((Nx,Ny))

s_x_old = np.zeros((Nx,Ny))
s_x_new = np.zeros((Nx,Ny))

s_y_old = np.zeros((Nx,Ny))
s_y_new = np.zeros((Nx,Ny))

## Variable conversion (C->P)

To switch from conservative to primitive variable we first note that,

$$S_iS^i/D^2 = 9(1-W^2)(3-W^2)^2$$

Now let $\Theta = S_iS^i/D^2$ and solving for $W^2$ we get,

$$W^2 = \frac{3}{2\Theta} \left(2 \Theta - 3 + \sqrt{9-8\Theta}\right)$$

With this,

$$e = \frac{D}{-1+3W^{-2}}$$

$$v^i = \frac{W^2 S^i}{3e}$$

In [4]:
def CtoP(D,Sx,Sy):
    
    Nx = S_x_old.shape[0]
    Nx = S_x_old.shape[1]
    prho = np.zeros((Nx,Ny))
    v_x  = np.zeros((Nx,Ny))
    v_y  = np.zeros((Nx,Ny))
    W2   = np.zeros((Nx,Ny))
    
    # Compute theta
    theta = (np.multiply(Sx,Sx)+np.multiply(Sy,Sy))/np.multiply(D,D)
    
    # Compute W^2
    W2 = (1.5/theta)*(2*theta-3.0+np.sqrt(9.0-8.0*theta))
    if np.any(np.isnan(W2)):
        print(np.amin(9.0-8.0*theta))
        print("Position: ",np.unravel_index(W2.argmin(), W2.shape)[0])
        print("Theta: ", theta[0,0])
        print("Sx: ",Sx[0,0])
        print("Sy: ",Sy[0,0])
        print("D: ",D[0,0])
    for i in range(Nx):
        for j in range(Ny):
            if (theta[i,j] < epsilon**2):
                W2[i,j] = 1.0
        
    # Compute primitives
    prho = D/(-1+(3/W2))
    
    v_x = (np.multiply(W2,Sx))/(3*prho)
    v_y = (np.multiply(W2,Sy))/(3*prho)
    
    return prho, v_x, v_y

def PtoC(rho,v_x,v_y,Nx,Ny):
    
    D = np.zeros((Nx,Ny))
    S_x = np.zeros((Nx,Ny))
    S_y = np.zeros((Nx,Ny))
    
    # Compute W^2
    W2 = np.ones((Nx,Ny))-(np.multiply(v_x,v_x)+np.multiply(v_y,v_y))
    
    D   = np.multiply(rho,-np.ones((Nx,Ny)) + 3.0/W2)
    S_x = 3.0*v_x/W2
    S_y = 3.0*v_y/W2
    
    return D, S_x, S_y

## Initial Data

In [5]:
def gen_init(dx,dy,Nx,Ny,x,y,plot=False):
    # initial data assumes square grid
    v_x = np.zeros((Nx,Ny))
    v_y = np.zeros((Nx,Ny))
    
    S_x = np.zeros((Nx,Ny))
    S_y = np.zeros((Nx,Ny))
    
    D = np.zeros((Nx,Ny))
    
    # Density of 1 everywhere
    rho = np.ones((Nx,Ny))
    
    W = np.zeros((Nx,Ny))
    W2 = np.zeros((Nx,Ny))
    
    # Generate velocities
    for i in range(Nx):
        if(y[i]>3 and y[i]<7):
            v_x[:,i] = 0.5*np.sin((2*np.pi)*y[i])*(y[i]-3.0)**2*((y[i]-7.0)**2)/16
        else:
            # Set to floor
            v_x[:,i] = epsilon
        if(x[i]>3 and x[i] < 7):
            v_y[i,:] = 0.5*np.sin((0.5*np.pi)*x[i])*(x[i]-3.0)**2*((x[i]-7.0)**2)/16
        else:
            # Set to floor
            v_y[i,:] = epsilon

    # Compute conservative variables
    D, S_x, S_y = PtoC(rho,v_x,v_y,Nx,Ny)

    if plot:
        fig = plt.figure(figsize=(15,15))
        ax = fig.add_subplot(111, projection='3d')
        X, Y = np.meshgrid(x, y)
        
        ax.plot_surface(X, Y, S_x)
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('S_x')
        
        plt.show()
    
    return v_x, v_y, rho, D, S_x, S_y

In [6]:
# Generate initial data
v_x_old, v_y_old, rho_old, D_old, S_x_old, S_y_old = gen_init(dx,dy,Nx,Ny,x,y)

### Define finite difference fn

In [7]:
# 2nd order Central difference for 1st degree derivative 
def dudx(u,dx):
    
    du = np.zeros((len(u),len(u)))
    
    # Assume periodic BC
    du[:,0]  = (u[1,:]-u[-2,:])/(2*dy)
    du[:,-1] = (u[1,:]-u[-2,:])/(2*dy)
    
    for i in range(1,len(u)-1):
        du[i,:] = (u[i+1,:]-u[i-1,:])/(2*dx)
        
    return du

# 2nd order Central difference for 1st degree derivative 
def dudy(u,dy):
    
    du = np.zeros((len(u),len(u)))
    
    # Assume periodic BC
    du[:,0]  = (u[:,1]-u[:,-2])/(2*dy)
    du[:,-1] = (u[:,1]-u[:,-2])/(2*dy)
    
    for i in range(1,len(u)-1):
        du[:,i] = (u[:,i+1]-u[:,i-1])/(2*dy)
        
    return du


### Define fluxes

In [8]:
def floor(u,Nx):
    for i in range(Nx):
        for j in range(Nx):
            if u[i,j] < epsilon:
                u[i,j] = epsilon
    return u

def fluxes(rho,v_x,v_y,D,S_x,S_y):
    
    Fx_rho = S_x
    Fy_rho = S_y
    
    Fx_Sx = rho+np.multiply(S_x,v_x)
    Fy_Sx = np.multiply(S_x,v_y)
    
    Fx_Sy = np.multiply(S_y,v_x)
    Fy_Sy = rho+np.multiply(S_y,v_y)
    
    return Fx_rho, Fy_rho, Fx_Sx, Fy_Sx, Fx_Sy, Fy_Sy

def RHS(Z):
    
    D   = Z[0]
    S_x = Z[1]
    S_y = Z[2]
    
    D = floor(D,Nx)
    
    rho, v_x, v_y =  CtoP(D,S_x,S_y)
    
    #print(np.amin(rho))
    rho = floor(rho,Nx)
    
    # Compute fluxes
    Fx_rho, Fy_rho, Fx_Sx, Fy_Sx, Fx_Sy, Fy_Sy = fluxes(rho,v_x,v_y,D,S_x,S_y)
        
    # Compute derivatives
    dxFx_rho = dudx(Fx_rho,dx)
    dyFy_rho = dudy(Fy_rho,dx)
    
    dxFx_Sx  = dudx(Fx_Sx,dx)
    dyFy_Sx  = dudy(Fy_Sx,dx)
    
    dxFx_Sy  = dudx(Fx_Sy,dx)
    dyFy_Sy  = dudy(Fy_Sy,dx)
    
    dD   = -dxFx_rho-dyFy_rho
    dS_x = -dxFx_Sx-dyFy_Sx
    dS_y = -dxFx_Sy-dyFy_Sy
        
    return np.array([dD, dS_x, dS_y])

### Define RK4

In [9]:
def rk4_step(f, y, h):
    
    k1 = h*f(y)
    k2 = h*f(y + 0.5 * k1)
    k3 = h*f(y + 0.5 * k2)
    k4 = h*f(y + k3)
    return y + (k1 + 2*k2 + 2*k3 + k4) / 6

### Evolve

In [10]:
# Create Z vector
Z_old = np.array([D_old,S_x_old,S_y_old])
t = [0]
X, Y = np.meshgrid(x, y)

for i in range(1,Nt+1):
    Z_new = rk4_step(RHS,Z_old,dt)
    t.append(t[0]+i*dt)
    print(i)
    # Save D figure
    plt.figure(figsize=(15,15))
    plt.contourf(X, Y, Z_new[0])
    plt.savefig("D" + str(i) + ".png")
    plt.close()
    
    #plt.figure(figsize=(15,15))
    #plt.contourf(X, Y, Z_new[1])
    #plt.savefig("Sx" + str(i) + ".png")
    #plt.close()
    
    #plt.figure(figsize=(15,15))
    #plt.contourf(X, Y, Z_new[2])
    #plt.savefig("Sy" + str(i) + ".png")
    #plt.close()
    
    Z_old = Z_new

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255


<ipython-input-4-5423a8cd2fdd>:14: RuntimeWarning: invalid value encountered in sqrt
  W2 = (1.5/theta)*(2*theta-3.0+np.sqrt(9.0-8.0*theta))


-1.8439430061466826
Position:  236
Theta:  4.506759582016501e-08
Sx:  0.0003001893908591225
Sy:  0.0003002610945284385
D:  2.0000000665174524
nan
Position:  235
Theta:  4.506753792677305e-08
Sx:  0.0003001900247122557
Sy:  0.00030026007565506164
D:  2.0000000680203325
nan
Position:  234
Theta:  4.507277341596394e-08
Sx:  0.00030021394436661217
Sy:  0.000300271036816548
D:  2.0000000792007397
256
nan
Position:  233
Theta:  4.507237038067587e-08
Sx:  0.0003002131615818725
Sy:  0.0003002691354898999
D:  2.000000080937232
nan
Position:  232
Theta:  4.5078074492016556e-08
Sx:  0.00030023895654637735
Sy:  0.0003002813409575165
D:  2.0000000931008883
nan
Position:  231
Theta:  4.507847759690102e-08
Sx:  0.000300244916963076
Sy:  0.00030027806592405654
D:  2.000000092337036
nan
Position:  230
Theta:  4.50843749378569e-08
Sx:  0.00030027955049970645
Sy:  0.0003002827164850051
D:  2.000000102676879
257
nan
Position:  229
Theta:  4.508446416353517e-08
Sx:  0.00030027669913896486
Sy:  0.0003002861

nan
Position:  143
Theta:  5.044192439985052e-08
Sx:  0.00033419808153051207
Sy:  0.0003001322628635049
D:  2.0000001750073952
nan
Position:  142
Theta:  5.132022301135667e-08
Sx:  0.0003394154953171795
Sy:  0.00030013006482921856
D:  2.0000002057199184
272
nan
Position:  141
Theta:  5.131188520019106e-08
Sx:  0.0003393695224446265
Sy:  0.0003001264912353347
D:  2.000000208064665
nan
Position:  140
Theta:  5.231235139743919e-08
Sx:  0.0003452206623157856
Sy:  0.00030012022625383
D:  2.000000240414733
nan
Position:  139
Theta:  5.244987829118237e-08
Sx:  0.00034601114249655987
Sy:  0.00030012639338847653
D:  2.000000236299008
nan
Position:  138
Theta:  5.364205647683852e-08
Sx:  0.00035283966039270664
Sy:  0.00030012073444475465
D:  2.0000002576445866
273
nan
Position:  137
Theta:  5.3632534403754565e-08
Sx:  0.0003527846110149225
Sy:  0.00030012199495140917
D:  2.0000002610707406
nan
Position:  136
Theta:  5.4993666320257905e-08
Sx:  0.00036042020227067213
Sy:  0.00030011998427740156
D

nan
Position:  78
Theta:  9.627765405192866e-07
Sx:  0.0019392534968841611
Sy:  0.00030069402257583216
D:  2.0000038581898756
288
nan
Position:  77
Theta:  9.626814556977646e-07
Sx:  0.00193915573784505
Sy:  0.00030069205371345163
D:  2.0000038600458914
nan
Position:  76
Theta:  1.0989955187472112e-06
Sx:  0.002074988728474196
Sy:  0.00030070273434374117
D:  2.0000041589634803
nan
Position:  75
Theta:  1.106870813812136e-06
Sx:  0.0020825665874520216
Sy:  0.0003006960585267711
D:  2.0000041683991228
nan
Position:  74
Theta:  1.2613167322235108e-06
Sx:  0.0022259529367646435
Sy:  0.000300704215050549
D:  2.0000044740390024
289
nan
Position:  73
Theta:  1.2612792840099108e-06
Sx:  0.0022259192756290806
Sy:  0.0003007043195152821
D:  2.000004474292021
nan
Position:  72
Theta:  1.4348879715552588e-06
Sx:  0.0023767951569083536
Sy:  0.00030070608593699724
D:  2.0000047881383716
nan
Position:  71
Theta:  1.443715968742955e-06
Sx:  0.002384212331878852
Sy:  0.00030070435193041184
D:  2.000004